In [1]:
from numpy.random import seed
seed(1337)
from tensorflow import set_random_seed
set_random_seed(2674)
import numpy as np
import h5py
import matplotlib
matplotlib.use('Qt4Agg')
import matplotlib.pyplot as plt
from scipy import integrate, interpolate, signal, optimize, stats
import cPickle as pickle
import lal
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, BatchNormalization, Flatten
from keras.optimizers import Nadam, SGD
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pyfftw
import progressbar
import time
from sklearn import metrics
import itertools
np.set_printoptions(edgeitems=30, linewidth=160)
import warnings
warnings.filterwarnings('ignore')
import pickle

Using TensorFlow backend.


In [ ]:
# This code is for reading simulated SNe waveforms
# This code will apply shift to the waveform 
# samples so that the waveform will always be in the certre +- user customized percentage.

In [2]:
# The name of the file that contains the simulated CCSN waveforms
filename = './Data/GWdatabase.h5'

# Read the simulated CCSN waveforms
waveformfile = h5py.File(filename, 'r')


# The first level keys of the h5 file
reduced_data = waveformfile.keys()[0]
waveformfilekey = waveformfile.keys()[1]
yeofrho = waveformfile.keys()[2]

waveformfamily = []
waveformfamily_keys = []

# Since there are 1824 different simulated CCSN waveform. 
# Each of which is saved in a different waveformfile key 
# So the loop below is to retreive all the keys with which the waveform strain data is accessed,
# and save it to waveformfamily.
# Each waveform family has 5 different keys, so the second part is to retrieve these 5 keys, and save them
# to waveformfamily_keys.

for i, key in enumerate(waveformfile[waveformfilekey].keys()):
    waveformfamily.append(key)
    if i == 0:
        for j, _ in enumerate(waveformfile[waveformfilekey][waveformfamily[i]].keys()):
            waveformfamily_keys.append(waveformfile[waveformfilekey][waveformfamily[i]].keys()[j])
originalSNR = np.array(waveformfile[reduced_data][u'SNR(aLIGOfrom10kpc)'])

[u'M_IC(Msun)', u'beta_IC', u'rho_c(g|ccm)', u'strain*dist(cm)', u't-tb(s)']

In [3]:
# This is to set some parameters for the training.
# Since the waveforms are stored in the unit of strain * distan
# the waveform amplitudes need to be divided by a distance.

# Convection factor between par sec and meters
PctMe = lal.PC_SI

# The distance the waveform will be divided by, in centimeters
Dist = 10.0 * 1e3 * PctMe * 1e2

# Since the waveform samples come in different lengths, 
# so every waveform sample will be set to the longest length.
# findmax/findmin is a variable that saves the longest/shortest length of the waveform samples.
# k/kmin is the index referring to the longest/shortest waveform sample.
findmax = 0
k = 0 
findmin = 1e10
kmin = 0
#length = np.zeros(len(waveformfamily))
#waveformfamily = [waveformfamily[0]]


# Since the waveform contains 1824 waveforms, which are different both in the morophology and the duration,
# training a network with all these waveforms may make it hard to debug. So one may want to limit the variation
# in the waveform samples by limiting the number of waveform samples put in the training. 
no_waves_considered = 50#1824
for i in range(len(waveformfamily[0:no_waves_considered])):
    waveformnumber = i

    ts = np.array(waveformfile[waveformfilekey][waveformfamily[waveformnumber]][u't-tb(s)']) 
    #waves = np.array(waveforms[waveformkey][waveformfamily[waveformnumber]][u'strain*dist(cm)']) / Dist 
    if findmax < len(ts):
        findmax = len(ts)
        k = i
    if findmin > len(ts):
        findmin = len(ts)
        kmin = i

print(findmax, k, findmin, kmin)


(20794, 40, 13188, 12)


In [4]:
# The simulated waveforms are sampled with a sampling rate equal to 65535 Hz, 
# coupled with the longest waveform is ~1.66s, this makes the longest waveform contains 1e5 elements. 
# Since this code will make other waveforms the same length as the longest length, this requires huge amount of memory,
# and makes training very slow and difficult. 
# Therefore, this codes uses scipy.signal.decimate to down sample the waveforms


def padandextractwave(waveformfile, waveformfilekey, waveformfamily, strainkey, wavemaxlength, Dist, no_waves_considered, R):
    # Number of simulated waveforms considered
    noofwaves = len(waveformfamily[0:no_waves_considered])
    
    msg = 'Reading waveforms from file and downsampling them by a factor of %s............' %(R)
    print(msg)
    bar = progressbar.ProgressBar(max_value = no_waves_considered)
    
    # downsample factor, the downsampled waveform will have length = original length / R
    
    # Vector used to save the downsampled waveform
    downsampled_waveforms = np.array([np.zeros(wavemaxlength / R) for i in range(noofwaves)])
    
    for i, whichsimulation in enumerate(waveformfamily[0:no_waves_considered]):
        
        # convert the unit of the waveform from strain*distance to strain
        wave = np.array(waveformfile[waveformfilekey][whichsimulation][strainkey]) / Dist
        wavelength = len(wave)
        
        # Pad the waveform with zero so that it has the same length as the longest waveform, 
        # or whatever length is set by wavemaxlength
        temporary = np.pad(wave, (0, wavemaxlength - wavelength), 'constant', constant_values = 0)
        
        # down sample
        downsampled_waveforms[i] = signal.decimate(temporary, R, ftype='iir')
        bar.update(i + 1)
        
    return downsampled_waveforms
    

In [5]:

# Since the original longest waveform length may not be dividable by the down sample vector, 
# this is to ensure that the length will be dividable. 
R = 8
findmax = 108512

findmax = np.ceil(findmax/8.0) * 8

# the assumed observation/simulation duration for every waveform 
Tobs = findmax / 65535.0
#start = time.time()
SNewaves = padandextractwave(waveformfile, waveformfilekey, waveformfamily, u'strain*dist(cm)', int(findmax), Dist, no_waves_considered, R)
#elapsed = time.time() - start
#print(elapsed)
# Using the downsampled waveform to compute the new sampling rate
New_sr = (len(SNewaves[0]) - 1) / Tobs
# the new spacing in time
New_dt = 1.0 / New_sr



  4% (2 of 50) |#                        | Elapsed Time: 0:00:00 ETA:  00:00:00

Reading waveforms from file and downsampling them by a factor of 8............


100% (50 of 50) |########################| Elapsed Time: 0:00:03 ETA:  00:00:00

In [6]:
def ASDtxt(x):
    """This function reads the following noise curves given a detector name."""
    return {
        'LET':'./ASD/ET_D.txt',
        'LCE':'./ASD/CE.txt',
        'H1': './ASD/ligoII_NS.txt',
        'L1': './ASD/ligoII_NS.txt',
        'V1': './ASD/virgoII.txt',
        'I2': './ASD/ligoII_NS.txt',
        'KAGRA': './ASD/ligoII_NS.txt',
        'ET_1': './ASD/ET_D.txt',
        'ET_2': './ASD/ET_D.txt',
        'ET_3': './ASD/ET_D.txt',
        'A2': './ASD/ligoII_NS.txt',
        'A2.5': './ASD/ligoII_NS.txt',
    }[x]


In [7]:
def readnos(detector, f_points):
    """This function interpolates the noise given the frequency samples."""
    nos_file = ASDtxt(detector)
    f_str = []
    ASD_str = []
    file = open(nos_file, 'r')
    readFile = file.readlines()
    file.close()
    f = []
    ASD = []
    
    for line in readFile:
        p = line.split()
        f_str.append(float(p[0]))
        ASD_str.append(float(p[1]))
    f = np.log10(np.array(f_str))
    ASD = np.log10(np.array(ASD_str))
    nosinterpolate = interpolate.splrep(f, ASD, w=1.0*np.ones(len(ASD)), s=0)
    
    nos = interpolate.splev(np.log10(f_points), nosinterpolate, der = 0, ext = 3)
    nos = 10**nos
    
    return nos

In [8]:
def noisegenerator(Tobs, det, SR, df, dt):
    """This function generates noise based on amplitude spectral density"""
    
    # The number of time stamps
    Ns = Tobs * SR 
    
    # The number of the frequency samples
    Nf = int(Ns // 2 + 1)
    
    # The frequency sample
    fs = np.arange(Nf) * df
    
    # read ASD
    ASD = readnos(det, fs)
    #plt.loglog(fs, ASD)
    #plt.show()
    #dd
    
    PSD = ASD ** 2
    # scale the ASD by the observation time, and this will be the highest amplitude of the generated noise
    Amp = np.sqrt(0.25 * Tobs * PSD)
    
    
    idx = np.argwhere(PSD==0.0)
    Amp[idx] = 0.0
    
    real_nos = Amp * np.random.normal(0.0, 1.0, Nf)
    img_nos = Amp * np.random.normal(0.0, 1.0, Nf)
    
    # This is to ensure there is no strange behaviour from noise at low frequency.
    # This is because the interpolation function will interpolate strange values at frequencies betweem 1 - 10Hz.
    #low_cutoff = 20
    #high_cutoff = 2048
    
    #idx_1 =  int(low_cutoff/df)
    #real_nos[0:idx_1] = 0
    #img_nos[0:idx_1] = 0
    #idx_2 = int(high_cutoff/df)
    #real_nos[idx_2:] = 0
    #img_nos[idx_2:] = 0
    
    nos = real_nos + 1j * img_nos

    
    # Fourier transiform converts the generated noise to the tme domain
    fftinput = pyfftw.empty_aligned(len(nos), dtype='complex128')
    
    fft_object = pyfftw.builders.irfft(fftinput)

    nos_realization = Ns* fft_object(nos) * df

    return ASD, nos_realization, fs
    

In [9]:
def SNR_calculator(waves_in_time_domain, dt, Det):
    
    length = len(waves_in_time_domain)
    
    df = 1.0 / (length * dt)
    
    Nf = int((length // 2 + 1))
    
    fftinput_for_snr = pyfftw.empty_aligned(length, dtype='complex128')     
    fft_object_for_snr = pyfftw.builders.rfft(fftinput_for_snr)      
     
    # frequency samples
    fs = np.arange(Nf) * df
    
    # Amplitude spectral density
    ASD = readnos(Det, fs)
        
    temporary_wave_in_f = fft_object_for_snr(waves_in_time_domain) * dt
    
    snr = np.sqrt( 4.0 * sum( abs(temporary_wave_in_f) ** 2 / ASD ** 2 ) * df )
    
    return snr
    

In [10]:
def rescale_to_set_SNR(preset_SNR, SNewaves, dt, Det):
    
    df = 1.0 / (len(SNewaves[0]) * dt)
    fftinput_for_snr = pyfftw.empty_aligned(len(SNewaves[0]), dtype='complex128')     
    fft_object_for_snr = pyfftw.builders.rfft(fftinput_for_snr)      
    
    Nf = int((len(SNewaves[0]) // 2 + 1))
    
    # frequency samples
    fs = np.arange(Nf) * df
    
    # Amplitude spectral density
    ASD = readnos(Det, fs)
    msg = 'Rescaling the amplitude of the waveforms so that their optimal SNR is %s.........' %(preset_SNR)
    print(msg)
    print(" ")
    bar = progressbar.ProgressBar(max_value = len(SNewaves))
    
    for i, wave in enumerate(SNewaves):
        temporary_wave_in_f = fft_object_for_snr(wave) * dt
        temporary_snr = np.sqrt( 4.0 * sum( abs(temporary_wave_in_f) ** 2 / ASD ** 2 ) * df )
        SNR_factor = preset_SNR / temporary_snr
    
        SNewaves[i] = SNR_factor * wave
        #print(temporary_snr)
        #print(  np.sqrt(4.0 * sum(abs(fft_object_for_snr(SNewaves[i]) * dt) **2 / ASD ** 2) * df))
        bar.update(i)
    
    return SNewaves
    

In [11]:
def data_generator(seed, ts, dt, Sr, percentage, Det, SNewaves, N_rz, multiplication):
    """This function generates the data for training/validation/testing."""
    
    np.random.seed(seed)
    
    # The number of sample will be equal to the number of N_rz(noise realizations)
    data = np.array([np.zeros_like(ts) for i in range(N_rz)])
    
    # Signal to noise ratio
    #SNR = np.zeros(N_rz)
    
    # Number of time stamps
    Ns = len(ts)
    
    # Number of frequency samples
    Nf = int(Ns //2 + 1)
    
    # Observation time
    Tobs = ts[-1] + dt
    
    # spacing in the frequency domain
    df = 1.0/Tobs
    # frequency samples
    fs = np.arange(Nf) * df
    
    # Amplitude spectral density
    ASD = readnos(Det, fs)
    
    
    toolbar_width = N_rz

    
    
    msg = 'Generating noise realizations.......'
    print(msg)
    print(" ")
    # setup toolbar
    bar = progressbar.ProgressBar(max_value=toolbar_width)
    

    
    # Generate noise
    for i in range(N_rz):
        #if (i+1) % 1000 == 0 & i != N_rz - 1:
        #   msg = 'The %s th to %s th noise realizations are now being generated.' %(i+1, i+1000)
        #    print(msg)
        _, data[i], _ = noisegenerator(Tobs, Det, Sr, df, dt)
        bar.update(i+1)



    msg = 'Adding noise to signals and converting them back to the time domain after whitening them in the frequency domain.....'
    print(msg)
    print(" ")
    bar_2 = progressbar.ProgressBar(max_value=toolbar_width)
    
    
    if ts[-1] == signal_duration:   

        for i in range(multiplication):
            for j in range(len(SNewaves)):

                count = i * len(SNewaves) + j
                #if (count + 1) % 1000 == 0 and count < 4999:
                #    msg = 'The %s th to %s th samples of the data set are now being generated.' %(count + 1,count + 1000)
                #    print(msg)
                data[count] += SNewaves[j]


                fftinput_1 = pyfftw.empty_aligned(len(data[count]), dtype='complex128')
                fft_object_1 = pyfftw.builders.rfft(fftinput_1)
                temporary = fft_object_1(data[count]) * 1.0/Sr
                temporary = temporary / ASD 


                #SNR[count] = np.sqrt(4.0 * sum(abs(temporary[int(100/df): int(500/df)]) ** 2 * df))
                #SNR_factor = SNR_set / SNR[count]
                #temporary = temporary * SNR_factor
                #if SNR_factor > 1:
                #    print(SNR_factor,count) 
                #print(SNR_factor, np.sqrt(4.0 * sum(abs(temporary) ** 2 * df)))
                fftinput_2 = pyfftw.empty_aligned(len(temporary), dtype='complex128')
                fft_object_2 = pyfftw.builders.irfft(fftinput_2)
                data[count] = Ns * fft_object_2(temporary) * df * np.sqrt(2.0/ Sr)
                bar_2.update( count + 1)
    elif ts[-1] > signal_duration:
        for i in range(multiplication):
            for j in range(len(SNewaves)):

                count = i * len(SNewaves) + j
                #if (count + 1) % 1000 == 0 and count < 4999:
                #    msg = 'The %s th to %s th samples of the data set are now being generated.' %(count + 1,count + 1000)
                #    print(msg)
                # This is to draw a random and determine     
                random_shift_percentage = np.random.uniform(-percentage, percentage)
                original_starting_point = sample_length / 2 - signal_length / 2
                shifted_starting_point = int(original_starting_point * (1 + random_shift_percentage))
                
                data[count][shifted_starting_point: shifted_starting_point + signal_length] = data[count][shifted_starting_point: shifted_starting_point + signal_length] + SNewaves[j]
        
                fftinput_1 = pyfftw.empty_aligned(len(data[count]), dtype='complex128')
                fft_object_1 = pyfftw.builders.rfft(fftinput_1)
                temporary = fft_object_1(data[count]) * 1.0 / Sr
                temporary = temporary / ASD 


                #SNR[count] = np.sqrt(4.0 * sum(abs(temporary[int(100/df): int(500/df)]) ** 2 * df))
                #SNR_factor = SNR_set / SNR[count]
                #temporary[int(100/df): int(500/df)] = temporary[int(100/df): int(500/df)] * SNR_factor
                
                #if SNR_factor > 1:
                #    print(SNR_factor,count) 
                #print(SNR_factor, np.sqrt(4.0 * sum(abs(temporary) ** 2 * df)))
                
                fftinput_2 = pyfftw.empty_aligned(len(temporary), dtype='complex128')
                fft_object_2 = pyfftw.builders.irfft(fftinput_2)
                data[count] = Ns * fft_object_2(temporary) * df * np.sqrt(2.0/ Sr)
                bar_2.update( count + 1 )
    else:
        raise Exception('The sample length should be longer than or equal to the signal length') 

            
    for i in range(multiplication * len(SNewaves), N_rz):
        fftinput_1 = pyfftw.empty_aligned(len(data[i]), dtype='complex128')
        fft_object_1 = pyfftw.builders.rfft(fftinput_1)
        temporary = fft_object_1(data[i]) *  1.0 / Sr 
        temporary = temporary / ASD 
        
        fftinput_2 = pyfftw.empty_aligned(len(temporary), dtype='complex128')
        fft_object_2 = pyfftw.builders.irfft(fftinput_2)
        data[i] = Ns * fft_object_2(temporary) * df * np.sqrt(2.0/ Sr)
        bar_2.update(i + 1)
            
            
    return data #SNR
        


In [12]:
def whiten_data(not_whitened_data_in_time_domain, ASD, dt, SR):
    
    num = len(not_whitened_data_in_time_domain)
    signal_len = len(not_whitened_data_in_time_domain[0])
    
    whitened_data = np.array([np.zeros(signal_len ) for i in range(num)])
    fftinput_in_td = pyfftw.empty_aligned(signal_len, dtype='complex128')
    fft_object_to_f = pyfftw.builders.rfft(fftinput_in_td)
    
    
    fftinput_in_fd = pyfftw.empty_aligned(signal_len//2 + 1, dtype='complex128')
    fft_object_to_t = pyfftw.builders.irfft(fftinput_in_fd)
    
    for i, nwd in enumerate(not_whitened_data_in_time_domain):
        temp = fft_object_to_f(nwd) * dt / ASD
        whitened_data[i] = fft_object_to_t(temp) * np.sqrt(2.0/ SR) / dt
        
    
    return whitened_data 



In [13]:
# the time stamps 
signal_length = len(SNewaves[0])
signal_duration = (signal_length - 1) * New_dt

# applying pad to make the sample longer. This is for the purpose of shifting the signal, so that the signal will appear to be in the centre +- user customised percentage
# If no padding is to be applied
sample_length = signal_length * 1.0

# time stamps after pad
ts = np.arange(sample_length) * New_dt
sample_duration = ts[-1]

In [14]:
SNR_set = 10.0
SNewaves = rescale_to_set_SNR(SNR_set, SNewaves, New_dt, 'H1')
plt.plot(SNewaves[0])
plt.show()

 84% (42 of 50) |####################    | Elapsed Time: 0:00:00 ETA:   0:00:00

Rescaling the amplitude of the waveforms so that their optimal SNR is 10.0.........
 


In [15]:
if 'data' in globals():
    del data
    del label
if 'train_sample' in globals():
    del train_sample, train_label, val_sample, val_label, test_sample, test_label
    
multiplication = 2
presence = len(SNewaves) * multiplication
shift_percentage = 0.0
seed = 10
Det = 'H1'

# Number of noise realization. This will be the final number of data samples for training + validation + testing
N_rz = 1000
data = data_generator(seed, ts, New_dt, New_sr, shift_percentage, Det, SNewaves, N_rz, multiplication)
label = np.concatenate((np.ones(presence), np.zeros(N_rz - presence)))

  2% (26 of 1000) |                      | Elapsed Time: 0:00:00 ETA:   0:00:06

Generating noise realizations.......
 


 11% (114 of 1000) |##                   | Elapsed Time: 0:00:00 ETA:   0:00:01

Adding noise to signals and converting them back to the time domain after whitening them in the frequency domain.....
 


 98% (988 of 1000) |#################### | Elapsed Time: 0:00:01 ETA:   0:00:00

In [16]:
asd, _, _ = noisegenerator(ts[-1] + New_dt , 'H1', New_sr, 1.0/signal_duration, New_dt)
whitened_data = whiten_data(SNewaves, asd, New_dt, New_sr)


In [16]:
i = np.random.randint(20000)
plt.plot(data[i])
#plt.plot(whitened_data[0])
print(label[i])
plt.show()

0.0


In [159]:
pathandname='showdata.pkl'
fp = open(pathandname,"w")
pickle.dump([ts, show_data], fp)
fp.close()

In [18]:

#plt.plot(ts, SNewaves[0])

plt.plot(ts, data[1])
#plt.plot(ts, val_sample[1773])
plt.plot(ts, whitened_data[1], color='r')

plt.show()
#plt.plot(data[1824])
#plt.show()

In [19]:
fig=plt.figure(figsize=(10, 8), dpi= 100, facecolor='w', edgecolor='k')
gd = stats.norm(0,1)
x = np.linspace(-10, 10, 1000)
y = gd.pdf(x)
plt.plot(x,y)
plt.hist(data[1], bins = 40, density=True)
plt.show()

In [16]:
def shuffle_data(sample, label,  shuffle_times, seed):
    np.random.seed(seed)
    for i in range(shuffle_times):
        state = np.random.randint(0,100)
        sample, label = shuffle(sample, label, random_state=state)
        
    return sample, label


In [17]:
def split_data(sample, label, savefortest, presence, shuffled, tv_split):
    
    data_size = len(sample) 
    
    
    if shuffled == True:
        split_point = int((data_size - 2 * savefortest) * tv_split)
        train_sample = sample[0 : split_point] 
        train_label = label[0 : split_point]
        
        val_sample = sample[split_point : data_size - 2 * savefortest]
        val_label = label[split_point : data_size - 2 * savefortest]
    
        test_sample = sample[data_size - 2 * savefortest : ]
        test_label = label[data_size - 2 * savefortest : ]

    else:

        nos_portion = (data_size - presence - savefortest)/2
        nos_start = presence

        
        train_start_index_signal = 0
        train_end_index_signal = (presence - savefortest) / 2

        train_start_index_noise = nos_start
        train_end_index_noise = nos_start + nos_portion


        val_start_index_signal = train_end_index_signal
        val_end_index_signal = presence - savefortest

        val_start_index_noise = train_end_index_noise
        val_end_index_noise = train_end_index_noise + nos_portion


        test_start_index_signal = val_end_index_signal
        test_end_index_signal =  val_end_index_signal + savefortest

        test_start_index_noise = val_end_index_noise
        test_end_index_noise =  val_end_index_noise + savefortest

    
        train_sample = np.concatenate((data[train_start_index_signal:train_end_index_signal], 
                                       data[train_start_index_noise:train_end_index_noise]))

        train_label = np.concatenate((np.ones(train_end_index_signal - train_start_index_signal), 
                                      np.zeros(train_end_index_noise - train_start_index_noise)))

        val_sample = np.concatenate((data[val_start_index_signal:val_end_index_signal], 
                                    data[val_start_index_noise:val_end_index_noise]))

        val_label = np.concatenate((np.ones(val_end_index_signal - val_start_index_signal), 
                                    np.zeros(val_end_index_noise - val_start_index_noise)))

        test_sample = np.concatenate((data[test_start_index_signal : test_end_index_signal], 
                                     data[test_start_index_noise : test_end_index_noise]))

        test_label = np.concatenate((np.ones(test_end_index_signal - test_start_index_signal), 
                                    np.zeros(test_end_index_noise - test_start_index_noise)))
        
        #print(train_start_index_signal,train_end_index_signal, train_start_index_noise , train_end_index_noise, 
        #      val_start_index_signal, val_end_index_signal, val_start_index_noise, val_end_index_noise,
        #      test_start_index_signal, test_end_index_signal, test_start_index_noise, test_end_index_noise)
    return train_sample, train_label, val_sample, val_label, test_sample, test_label

In [79]:
plt.plot(data[3647])
plt.show()

In [23]:
len(val_sample)

2960

In [24]:
print(len(train_sample))
print(len(val_sample))
print(len(test_sample))
i = 10
plt.plot(test_sample[i])
plt.show()
print(test_label[i])


11840
2960
200
1.0


In [18]:
presence = len(SNewaves) * multiplication
label = np.concatenate((np.ones(presence), np.zeros(N_rz - presence)))

shuffle_times = 1
seed_s = 387
sample, label = shuffle_data(data, label,  shuffle_times, seed_s)
savefortest = 100
shuffled = True
tv_split = 0.8
train_sample, train_label, val_sample, val_label, test_sample, test_label = split_data(sample, label, savefortest, presence, shuffled, tv_split)
del sample

In [161]:
i = 2
plt.plot(val_sample[i])
plt.show()
print(val_label[i])

0.0


In [139]:
def checkvalidity(data, label):
    co = 0
    for la, sa in zip(label, data):
        if la == 0 and np.max(abs(sa)) >= 5:
            print(co)
        elif la == 1 and np.max(abs(sa)) < 5:
            print(co)
        co += 1


In [159]:
checkvalidity(test_sample, test_label)

In [19]:
"""Below is the CNN part of this code""" 

batch_size = 30      # number of time series per batch
num_classes = 2      # signal or background
epochs = 20          # number of full passes of the dataset
outdir = './results' # directory to store results in



In [20]:
number_of_sample_for_training = len(train_sample)
number_of_sample_for_testing = len(test_sample)
number_of_sample_for_validation = len(val_sample)
training_sample_length = len(train_sample[0])

train_sample = train_sample.reshape(number_of_sample_for_training, 1, training_sample_length)
test_sample = test_sample.reshape(number_of_sample_for_testing, 1, training_sample_length)
val_sample = val_sample.reshape(number_of_sample_for_validation, 1, training_sample_length)

In [21]:
keras.backend.set_image_data_format('channels_first')
training_sample_length = len(train_sample[0][0])

train_sample = train_sample.reshape(-1, 1, 1, training_sample_length)
val_sample = val_sample.reshape(-1, 1, 1, training_sample_length)
test_sample = test_sample.reshape(-1, 1, 1, training_sample_length)

input_shape = train_sample.shape[1:]

In [24]:
plt.plot(train_sample[0][0][0])
plt.show()

In [22]:
train_label = keras.utils.to_categorical(train_label , num_classes)
val_label = keras.utils.to_categorical(val_label, num_classes)
test_label = keras.utils.to_categorical(test_label, num_classes)

In [23]:
del model


NameError: name 'model' is not defined

In [24]:
model = Sequential()    # define the type of keras model

# add the layers
# conv1
model.add(Conv2D(8, (1,64), activation='elu', input_shape=input_shape))
# maxpool2
model.add(MaxPool2D((1,8)))
# conv2
model.add(Conv2D(16, (1,16), activation='elu'))
# maxpool2
model.add(MaxPool2D((1,6)))
# the input the fully connected layer must be 1-D vector
model.add(Flatten())
model.add(Dense(32, activation='elu'))
#model.add(Dropout(0.5))
dol = keras.layers.Dropout(0.5, noise_shape=None, seed=10)

model.add(dol)
# add the output layer with softmax actiavtion for classication
model.add(Dense(num_classes, activation='softmax'))
# print a summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 8, 1, 13501)       520       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 1, 1687)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 1, 1672)       2064      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 1, 278)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4448)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                142368    
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
__________

In [48]:
del model


In [49]:
model = Sequential()    # define the type of keras model

# add the layers
# conv1
model.add(Conv2D(16, (1,64), activation='elu', input_shape=input_shape))
# maxpool2
model.add(MaxPool2D((1,8)))


model.add(Conv2D(16, (1,16), activation='elu'))
# maxpool2
model.add(MaxPool2D((1,6)))


# the input the fully connected layer must be 1-D vector
model.add(Flatten())
model.add(Dense(8, activation='elu'))

dol = keras.layers.Dropout(0.5, noise_shape=None, seed=10)

model.add(dol)
# add the output layer with softmax actiavtion for classication
model.add(Dense(num_classes, activation='softmax'))
# print a summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 16, 1, 13501)      1040      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 1, 1687)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 1, 1672)       4112      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 16, 1, 278)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 4448)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 35592     
_________________________________________________________________
dropout_5 (Dropout)          (None, 8)                 0         
__________

In [25]:
# compile the model
#adam = keras.optimizers.Adam(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer= Nadam(),
              metrics=['accuracy'])

In [26]:
# save the model each epoch if it is the best so far
weight_file_name = '{0}/best_weights_at_SNR_%s.hdf5' %(int(SNR_set))
modelCheck = ModelCheckpoint(weight_file_name.format(outdir), monitor='val_acc', verbose=0, 
                save_best_only=True,save_weights_only=True, mode='auto', period=0)

In [28]:
history = model.fit(train_sample, train_label,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(val_sample, val_label),
                    callbacks = [modelCheck],
                   shuffle = False)

Train on 640 samples, validate on 160 samples
Epoch 1/20


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@training/Nadam/gradients/conv2d_1/convolution_grad/Conv2DBackpropFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_conv2d_1_input_0_0/_67, conv2d_1/kernel/read)]]
	 [[{{node metrics/acc/Mean/_99}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_838_metrics/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [62]:
if "history_saver" in globals():
    ##history_saver = []
    history_saver.append(history)
else:
    with open("history_saver.pkl") as f:
        history_saver = pickle.load(f)
        history_saver.append(history)

#pathandname='showdata.pkl'
#fp = open(pathandname,"w")
#pickle.dump([ts, show_data], fp)
#fp.close()

In [63]:
history_file_name = 'results/history_at_SNR_%s.pkl' %(int(SNR_set))
#pathandname = history_file_name
fp = open(history_file_name,"w")
pickle.dump([history_saver], fp)
fp.close()

In [66]:
weight_file_name.format(outdir)

'./results/best_weights_at_SNR_8.hdf5'

In [ ]:
# load the best model
weight_file_name = '{0}/best_weights_at_SNR_%s.hdf5' %(int(8))

model.load_weights(weight_file_name.format(outdir))
# evaluate
eval_results = model.evaluate(test_sample, test_label,
                              sample_weight=None,
                              batch_size=batch_size, verbose=1)
print('Test loss:', eval_results[0])
print('Test accuracy:', eval_results[1])
signal_preds = model.predict(test_sample)


In [ ]:
8 200/200 [==============================] - 0s 491us/step
('Test loss:', 0.186470251949504)
('Test accuracy:', 0.9599999845027923)

9 200/200 [==============================] - 0s 375us/step
('Test loss:', 0.08477069551008753)
('Test accuracy:', 0.9849999874830246)

10 200/200 [==============================] - 0s 327us/step
('Test loss:', 0.045048040983525085)
('Test accuracy:', 0.9949999958276748)

In [48]:
fontsize = 20

fig , axs = plt.subplots(2,1, sharex = True)
axs = axs.ravel()
# plot history
axs[0].plot(history.history['loss'], label = 'Loss', linewidth = 2, color = 'b')
axs[0].plot(history.history['val_loss'], label = 'Validation Loss', linewidth = 2, color = 'r')
axs[1].plot(history.history['acc'], label = 'Accuracy', linewidth = 2, color = 'b')
axs[1].plot(history.history['val_acc'], label = 'Validation Accurarcy', linewidth = 2, color = 'r')
# set labels
axs[0].set_ylabel('Loss', fontsize = fontsize)
axs[1].set_xlabel('Epoch', fontsize = fontsize)
axs[1].set_ylabel('Acc', fontsize = fontsize)
# legends
axs[0].legend(fontsize = fontsize)
axs[1].legend(fontsize = fontsize)
# grids
axs[0].grid()
axs[1].grid()
axs[0].set_xlim([0, epochs])
axs[0].set_ylim(bottom = 0)

axs[1].set_xlim([0, epochs])
axs[1].set_ylim(top = 1)

plt.subplots_adjust(left = 0.1, bottom = 0.1, right = 0.90, top = 0.95)
for ax in axs:
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontweight('normal')
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontweight('normal')
plt.show()

In [49]:
fa, ta, _ = metrics.roc_curve(test_label[:,1], signal_preds[:,1])
fig = plt.figure()
plt.plot(fa, ta, linewidth = 2, color = 'b')
plt.xlabel('False alarm probability',fontsize = fontsize)
plt.ylabel('True alarm probability',fontsize = fontsize)
plt.title('ROC curve for SNR %s'%(SNR_set), fontsize = fontsize)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.subplots_adjust(left = 0.1, bottom = 0.1, right = 0.90, top = 0.95)

plt.grid()
ax = plt.gca()
for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
plt.show()

In [64]:
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

import numpy as np

def get_demo_image():
    from matplotlib.cbook import get_sample_data
    import numpy as np
    f = get_sample_data("axes_grid/bivariate_normal.npy", asfileobj=False)
    z = np.load(f)
    # z is a numpy array of 15x15
    return z, (-3,4,-4,3)

fig, ax = plt.subplots(figsize=[5,4])

# prepare the demo image
Z, extent = get_demo_image()
Z2 = np.zeros([150, 150], dtype="d")
ny, nx = Z.shape
Z2[30:30+ny, 30:30+nx] = Z

# extent = [-3, 4, -4, 3]
ax.imshow(Z2, extent=extent, interpolation="nearest",
          origin="lower")

axins = zoomed_inset_axes(ax, 6, loc=1) # zoom = 6
#axins.imshow(Z2, extent=extent, interpolation="nearest",
#             origin="lower")

# sub region of the original image
x1, x2, y1, y2 = -1.5, -0.9, -2.5, -1.9
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

plt.xticks(visible=False)
plt.yticks(visible=False)

# draw a bbox of the region of the inset axes in the parent axes and
# connecting lines between the bbox and the inset axes area
#mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")

plt.draw()
plt.show()

In [118]:
## plotting SNR
fontsize = 26
plt.scatter(np.arange(1824)+1, originalSNR, color='b', label = 'Waveform SNR from waveform file')
plt.scatter(np.arange(1824)+1, SNR, color = 'r', label='SNR as computed using the optimal SNR formula')
plt.xlabel('Index of waveform samples', fontsize=fontsize)
plt.ylabel('SNR for signal from 100Hz to 500hz', fontsize =fontsize)
plt.grid()
plt.legend(fontsize=fontsize)
ax = plt.gca()
for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
plt.show()
    

In [ ]:
# plotting waveforms between original and downsampled
from mpl_toolkits.axes_grid.inset_locator import inset_axes

plt.plot(np.arange(len(original[0])) * original_dt, original[0], linewidth = 2, color = 'b', label = 'Original')

plt.plot(np.arange(len(SNewaves[0])) * New_dt, SNewaves[0], linewidth = 2, color = 'r', label = 'Downsampled by a factor of 8')
plt.legend()
fontsize =26
plt.xlabel('Time (s)', fontsize = fontsize)
plt.ylabel('Strain', fontsize =fontsize)
plt.legend(fontsize = fontsize, loc = 'lower right')
plt.xlim([0.16, 0.3])

plt.grid()

plt.show()

ax = plt.gca()

for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
text = ax.yaxis.get_offset_text()
text.set_size(fontsize)


ax = plt.gca()
inset_axes = inset_axes(ax, width="40%", # width = 30% of parent_bbox
                    height=2.0, # height : 1 inch
                    loc=1)
plt.grid()
plt.plot(np.arange(len(original[0])) * original_dt, original[0], linewidth = 2, color = 'b', label = 'Original')

plt.plot(np.arange(len(SNewaves[0])) * New_dt, SNewaves[0], linewidth = 2, color = 'r', label = 'Downsampled by a factor of 8')
plt.xlim([0.165, 0.195])
plt.ylim([-4e-22, 3e-22])

x = [0.165, 0.180, 0.195]
y = [-1.75e-22, -1.5e-22, -1.25e-22, -1.0e-22]

plt.xticks(x)


for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
text = ax.yaxis.get_offset_text()
text.set_size(fontsize)

In [121]:
plt.plot(np.arange(len(data[12])) * New_dt, data[12], linewidth = 2, color = 'b', label = 'Whitened signal example')
plt.legend()
fontsize =26
plt.xlabel('Time (s)', fontsize = fontsize)
plt.ylabel('Strain', fontsize =fontsize)
plt.legend(fontsize = fontsize, loc = 'upper right')
#plt.xlim([0.16, 0.3])

plt.grid()

plt.show()

ax = plt.gca()

for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
text = ax.yaxis.get_offset_text()
text.set_size(fontsize)
plt.show()




In [26]:

fontsize =22
fig , axs = plt.subplots(2,1, sharex = True)
axs = axs.ravel()
# plot history
axs[0].plot(history.history['loss'], linewidth =2, color ='r', label = 'loss')
axs[0].plot(history.history['val_loss'], linewidth =2, color ='b',label = 'val loss')
axs[1].plot(history.history['acc'], linewidth =2, color ='r',label = 'acc')
axs[1].plot(history.history['val_acc'], linewidth =2, color ='b',label = 'val_acc')
# set labels
axs[0].set_ylabel('Loss', fontsize = fontsize)
axs[1].set_xlabel('Epoch', fontsize = fontsize)
axs[1].set_ylabel('Acc', fontsize = fontsize)
# legends
axs[0].legend()
axs[1].legend()
# grids
axs[0].grid()
axs[1].grid()


ax = axs[0]

for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
text = ax.yaxis.get_offset_text()
text.set_size(fontsize)


ax = axs[1]

for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
text = ax.yaxis.get_offset_text()
text.set_size(fontsize)





plt.show()

In [ ]:

toolbar_width = 10000/100

# setup toolbar
bar = progressbar.ProgressBar(max_value=10000)

# Generate noise
for i in range(10000):
    #if (i+1) % 1000 == 0 & i != N_rz - 1:
    #    msg = 'The %s th to %s th noise realizations are now being generated.' %(i+1, i+1000)
    #    print(msg)
    #data[i], _ = noisegenerator(Tobs, Det, Sr, df, dt)
    #print((i+1)%100)
    if (i+1) % 100 == 0:
    # update the bar
        print(( float(i+1) / N_rz))
        bar.update( int(float(i+1)))


In [223]:
import pycbc.noise
import pycbc.psd
import pylab

# The color of the noise matches a PSD which you provide
flow = 30.0
delta_f = 1.0 / 32
flen = int(2048 / delta_f) + 1
psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

# Generate 32 seconds of noise at 4096 Hz
delta_t = 1.0 / 4096
tsamples = int(32 / delta_t)
ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd, seed=127)

pylab.plot(ts.sample_times, ts)
pylab.ylabel('Strain')
pylab.xlabel('Time (s)')
pylab.show()

In [241]:
plt.plot(np.fft.irfft(np.fft.rfft(np.array(ts))[960:65535]*delta_t/np.sqrt(np.array(psd)[960:65535])) /delta_t)
plt.show()

In [233]:
save_for_test = 100
nos_portion = (len(data) - multiplication * len(SNewaves) - save_for_test)/2


nos_start = multiplication * len(SNewaves)

train_start_index_signal = 0
train_end_index_signal = (multiplication * len(SNewaves) - save_for_test) / 2

train_start_index_noise = nos_start
train_end_index_noise = nos_start + nos_portion


val_start_index_signal = train_end_index_signal
val_end_index_signal = multiplication * len(SNewaves) - save_for_test

val_start_index_noise = train_end_index_noise
val_end_index_noise = train_end_index_noise + nos_portion


test_start_index_signal = val_end_index_signal
test_end_index_signal =  val_end_index_signal + save_for_test

test_start_index_noise = val_end_index_noise
test_end_index_noise =  val_end_index_noise + save_for_test


train_sample = np.concatenate((data[train_start_index_signal:train_end_index_signal], 
                               data[train_start_index_noise:train_end_index_noise]))

train_label = np.concatenate((np.ones(train_end_index_signal - train_start_index_signal), 
                              np.zeros(train_end_index_noise - train_start_index_noise)))

val_sample = np.concatenate((data[val_start_index_signal:val_end_index_signal], 
                            data[val_start_index_noise:val_end_index_noise]))

val_label = np.concatenate((np.ones(val_end_index_signal - val_start_index_signal), 
                            np.zeros(val_end_index_noise - val_start_index_noise)))


test_sample = np.concatenate((data[test_start_index_signal : test_end_index_signal], 
                             data[test_start_index_noise : test_end_index_noise]))

test_label = np.concatenate((np.ones(test_end_index_signal - test_start_index_signal), 
                            np.zeros(test_end_index_noise - test_start_index_noise)))


array([65535])

In [ ]:
def shuffle_data(train_sample, train_label, val_sample, val_label, test_sample, test_label,  shuffle_times):
    
    for i in range(shuffle_times):
        state = np.random.randint(0,100)
        train_sample, train_label = shuffle(train_sample, train_label, random_state=state)

        state = np.random.randint(0,100)
        test_sample, test_label = shuffle(test_sample, test_label, random_state=state)

        state = np.random.randint(0,100)
        val_sample, val_label = shuffle(val_sample, val_label, random_state=state)

    return train_sample, train_label, val_sample, val_label, test_sample, test_label
train_sample, train_label, val_sample, val_label, test_sample, test_label = shuffle_data(train_sample, train_label, val_sample, val_label, test_sample, test_label,  5)